<a href="https://colab.research.google.com/github/Emivk/RNA-Digits/blob/master/RNAdigit_tf_comet_mejora1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.9/561.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.2
    Uninstalling websocket-client-1.6.2:
      Successfully uninstalled websocket-client-1.6.2
  Attempting uninstall: python-box
    Found existing installation: python-box 7.1.1
    Uninstalling python-box-7.1.1:
      Successfully uninstalled python-box-7.1.1


In [20]:
import comet_ml
comet_ml.init(project_name="rnadigits")

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint

In [24]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/emivk/rnadigits/e1b395c7c0b74c90b7a506088e533fac
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is 

In [25]:
dataset=mnist.load_data()
(x_train, y_train), (x_test, y_test) = dataset

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255  # x_trainv = x_trainv/255
x_test /= 255

num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [26]:
parameters = {
    "batch_size": 100,
    "epochs": 50,
    "optimizer": "RMSprop",
    "loss": "categorical_crossentropy",
}

experiment.log_parameters(parameters)

In [27]:
model = Sequential()
model.add(Input(shape=(28,28))) #Flaten no tiene la opcion input_shape por lo tanto se tiene que agregar esta capa
model.add(Flatten()) #Otra forma de aplanar las imagenes
model.add(Dense(100, activation='relu'))
#model.add(Dense(400, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))) #Regularizacion L1L2
#model.add(Dropout(0.2)) #Fraccion de enlaces a eliminar
#model.add(Dense(200)) #Capa lineal , transformacion lineal sin funcion de activacion
#model.add(Activation('tanh')) #Se puede agregar despues la funcion de activacion
#model.add(Dense(400, activation='selu', kernel_regularizer=regularizers.L1(0.01) )) #Regularizacion L1
#model.add(Dense(200, activation='elu', kernel_regularizer=regularizers.L2(l2=1e-4)) ) #Regularizacion L2
#model.add(Dense(50,activation='exponential'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# specify the path where you want to save the model
filepath = "mejor-modelo1.2.hdf5"

# initialize the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [29]:
model.compile(loss=parameters['loss'],optimizer=parameters['optimizer'],metrics=['accuracy'])
model.fit(x_train, y_trainc,
                    batch_size=parameters['batch_size'],
                    epochs=parameters["epochs"],
                    verbose=1,
                    validation_data=(x_test, y_testc),
                    callbacks=[checkpoint])

Epoch 1/50
587/600 [============================>.] - ETA: 0s - loss: 0.3336 - accuracy: 0.9077
Epoch 1: val_loss improved from inf to 0.18817, saving model to mejor-modelo1.2.hdf5
600/600 [==============================] - 4s 5ms/step - loss: 0.3300 - accuracy: 0.9086 - val_loss: 0.1882 - val_accuracy: 0.9440
Epoch 2/50
597/600 [============================>.] - ETA: 0s - loss: 0.1612 - accuracy: 0.9529
Epoch 2: val_loss improved from 0.18817 to 0.13514, saving model to mejor-modelo1.2.hdf5
600/600 [==============================] - 4s 6ms/step - loss: 0.1610 - accuracy: 0.9530 - val_loss: 0.1351 - val_accuracy: 0.9612
Epoch 3/50
599/600 [============================>.] - ETA: 0s - loss: 0.1176 - accuracy: 0.9657
Epoch 3: val_loss improved from 0.13514 to 0.10478, saving model to mejor-modelo1.2.hdf5
600/600 [==============================] - 3s 5ms/step - loss: 0.1175 - accuracy: 0.9657 - val_loss: 0.1048 - val_accuracy: 0.9688
Epoch 4/50
589/600 [============================>.] - ET

In [30]:
experiment.log_model("MNIST1", "mejor-modelo1.1.hdf5")

{'web': 'https://www.comet.com/api/asset/download?assetId=6853f711258c43beb571530a4d1719a8&experimentKey=68e6c9e21c3142e2aa61c59f6a326d7e',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=6853f711258c43beb571530a4d1719a8&experimentKey=68e6c9e21c3142e2aa61c59f6a326d7e',
 'assetId': '6853f711258c43beb571530a4d1719a8'}

In [31]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/emivk/rnadigits/68e6c9e21c3142e2aa61c59f6a326d7e
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [50]                 : (0.9086166620254517, 1.0)
COMET INFO:     batch_accuracy [3000]         : (0.09000000357627869, 1.0)
COMET INFO:     batch_loss [3000]             : (6.43274761387147e-05, 2.439486503601074)
COMET INFO:     epoch_duration [50]           : (2.4658994330002315, 4.254926769999656)
COMET INFO:     loss [50]                     : (0.00013807039067614824, 0.33002814650535583)
COMET INFO:     val_accuracy [50]             : (0.9440000057220459, 0.9796000123023987)
COMET INFO:     val_loss [50]      